<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 37px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Machine Learning<br>
  </div> 

<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  -<br>
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Illustration with Random Forests<br><br>
  </div> 

<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      <font color=orange> 2 - Data preprocessing </font>
  </div>

<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      EC Lyon - T4 2024
  </div>

<a id="plan"></a>

## Data Preprocessing

-  [About the dataset and Kaggle](#about)<br>
-  [Load the data](#load_data)<br>
-  [Look at the data](#look_data)<br>
-  [Look at the columns](#look_columns)<br>
-  [Variable to predict](#output_variable)<br>
-  [Naive first model training](#naive_training)<br>
-  [Convert date times](#convert_date)<br>
-  [Convert strings into numerotated categories](#convert_strings)<br>
-  [Inspect missing values](#nans)<br>
-  [Save preprocessed data](#save_data)<br>
-  [Fully numericalize data](#numericalize_data)<br>



### Imports

In [1]:
# for package auto reload
%load_ext autoreload
%autoreload 2

# for better rendering of plots in jupyter notebook
%matplotlib inline

In [2]:
# base modules
from pathlib import Path

# for manipulating data
import numpy as np
import pandas as pd

# forMachine Learning
from sklearn.ensemble import RandomForestRegressor

In [3]:
from eclyon.transforms import (
    add_date_columns, change_columns_from_str_to_categorical, process_df,
)

# Data preprocessing

[Back to top](#plan)

<a id="about"></a>

### $\bullet$ About the dataset and Kaggle

[Back to top](#plan)

We will be looking at the Blue Book for Bulldozers Kaggle Competition: "The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuration.  The data is sourced from auction result postings and includes information on usage and equipment configurations."

This is a very common type of dataset and prediciton problem, and similar to what you may see in your project or workplace.

Kaggle is an awesome resource for aspiring data scientists or anyone looking to improve their machine learning skills.  There is nothing like being able to get hands-on practice and receiving real-time feedback to help you improve your skills.

Kaggle provides:

1. Interesting data sets
2. Feedback on how you're doing
3. A leader board to see what's good, what's possible, and what's state-of-art.
4. Blog posts by winning contestants share useful tips and techniques.



Kaggle provides info about some of the fields of our dataset; on the [Kaggle Data info](https://www.kaggle.com/c/bluebook-for-bulldozers/data) page they say the following:

For this competition, you are predicting the sale price of bulldozers sold at auctions. The data for this competition is split into three parts:

- **Train.csv** is the training set, which contains data through the end of 2011.
- **Valid.csv** is the validation set, which contains data from January 1, 2012 - April 30, 2012. You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
- **Test.csv** is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.

<a id="load_data"></a>

### $\bullet$ Import the data

[Back to top](#plan)

In [4]:
path_to_repo = Path('..').resolve()
path_to_data = path_to_repo / 'data'

In [5]:
df_raw = pd.read_csv(path_to_data / 'Train.csv', low_memory = False, parse_dates = ["saledate"])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/naomi/Desktop/EM/MachineLearning/ml-project/ECLyon-ML-Course-2024-T4/data/Train.csv'

In [ ]:
df_raw = df_raw.convert_dtypes()

<a id="look_data"></a>

### $\bullet$ Look at the data

[Back to top](#plan)

In any sort of data science work, it's **important to look at your data**, to make sure you understand the format, how it's stored, what type of values it holds, etc.<br> 
Even if you've read descriptions about your data, the actual data may not be what you expect.

The key fields that are in train.csv are:

- SalesID: the unique identifier of the sale
- MachineID: the unique identifier of a machine.  A machine can be sold multiple times
- saleprice: what the machine sold for at auction (only provided in train.csv)
- saledate: the date of the sale

In [7]:
# print the 5 first and last rows
df_raw

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3,2004,68,Low,2006-11-16,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
1,1139248,57000,117657,77,121,3,1996,4640,Low,2004-03-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
2,1139249,10000,434808,7009,121,3,2001,2838,High,2004-02-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,1139251,38500,1026470,332,121,3,2001,3486,High,2011-05-19,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,1139253,11000,1057373,17311,121,3,2007,722,Medium,2009-07-23,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1,2005,<NA>,<NA>,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
401121,6333337,11000,1830472,21439,149,1,2005,<NA>,<NA>,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
401122,6333338,11500,1887659,21439,149,1,2005,<NA>,<NA>,2011-11-02,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>
401123,6333341,9000,1903570,21435,149,2,2005,<NA>,<NA>,2011-10-25,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,<NA>,<NA>,<NA>,<NA>,<NA>


In [8]:
# print the first n rows (default is n = 5)
#df_raw.head(n = 3)
df_raw.head(3)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3,2004,68,Low,2006-11-16,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
1,1139248,57000,117657,77,121,3,1996,4640,Low,2004-03-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Standard,Conventional
2,1139249,10000,434808,7009,121,3,2001,2838,High,2004-02-26,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
help(df_raw.head)

Help on method head in module pandas.core.generic:

head(n: 'int' = 5) -> 'Self' method of pandas.core.frame.DataFrame instance
    Return the first `n` rows.
    
    This function returns the first `n` rows for the object based
    on position. It is useful for quickly testing if your object
    has the right type of data in it.
    
    For negative values of `n`, this function returns all rows except
    the last `|n|` rows, equivalent to ``df[:n]``.
    
    If n is larger than the number of rows, this function returns all rows.
    
    Parameters
    ----------
    n : int, default 5
        Number of rows to select.
    
    Returns
    -------
    same type as caller
        The first `n` rows of the caller object.
    
    See Also
    --------
    DataFrame.tail: Returns the last `n` rows.
    
    Examples
    --------
    >>> df = pd.DataFrame({'animal': ['alligator', 'bee', 'falcon', 'lion',
    ...                    'monkey', 'parrot', 'shark', 'whale', 'zebra']})
    >

<a id="look_columns"></a>

### $\bullet$ Look at the columns

[Back to top](#plan)

In [10]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [11]:
len(df_raw.columns)

53

In [12]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [13]:
df_raw.shape

(401125, 53)

In [14]:
display_all(df_raw.head().T)

,0,1,2,3,4
SalesID,1139246,1139248,1139249,1139251,1139253
SalePrice,66000,57000,10000,38500,11000
MachineID,999089,117657,434808,1026470,1057373
ModelID,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneerID,3,3,3,3,3
YearMade,2004,1996,2001,2001,2007
MachineHoursCurrentMeter,68,4640,2838,3486,722
UsageBand,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [15]:
display_all(df_raw.describe(include = 'all').T)

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
SalesID,401125.0,<NA>,<NA>,<NA>,1919712.521503,1139246.0,1418371.0,1639422.0,2242707.0,6333342.0,909021.492667
SalePrice,401125.0,<NA>,<NA>,<NA>,31099.712848,4750.0,14500.0,24000.0,40000.0,142000.0,23036.898502
MachineID,401125.0,<NA>,<NA>,<NA>,1217902.517971,0.0,1088697.0,1279490.0,1468067.0,2486330.0,440991.954249
ModelID,401125.0,<NA>,<NA>,<NA>,6889.70298,28.0,3259.0,4604.0,8724.0,37198.0,6221.777842
datasource,401125.0,<NA>,<NA>,<NA>,134.66581,121.0,132.0,132.0,136.0,172.0,8.962237
auctioneerID,380989.0,<NA>,<NA>,<NA>,6.55604,0.0,1.0,2.0,4.0,99.0,16.976779
YearMade,401125.0,<NA>,<NA>,<NA>,1899.156901,1000.0,1985.0,1995.0,2000.0,2013.0,291.797469
MachineHoursCurrentMeter,142765.0,<NA>,<NA>,<NA>,3457.955353,0.0,0.0,0.0,3025.0,2483300.0,27590.256413
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,NaN,NaN,NaN,2004-08-02 07:47:13.986413184,1989-01-17 00:00:00,2000-10-07 00:00:00,2006-03-23 00:00:00,2009-03-24 00:00:00,2011-12-30 00:00:00,NaN


<a id="output_variable"></a>

### $\bullet$ Variable to predict

[Back to top](#plan)

The variable we want to predict is the sale price of a bulldozer, based on its description. This means that we seek to **predict** values within the _SalePrice_ column of the dataset, by using the other columns as **expainatory variables** for prediction.

It's important to note what metric is being used for a project. Generally, selecting the metric(s) is an important part of the project setup. However, in this case Kaggle tells us what metric to use: RMSLE (root mean squared log error) between the actual and predicted auction prices. Therefore we take the log of the prices, so that RMSE will give us what we need.

In [16]:
df_raw.SalePrice

0         66000
1         57000
2         10000
3         38500
4         11000
          ...  
401120    10500
401121    11000
401122    11500
401123     9000
401124     7750
Name: SalePrice, Length: 401125, dtype: Int64

In [17]:
sale_prices = df_raw.SalePrice.tolist()

In [18]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [19]:
df_raw.SalePrice

0          11.09741
1         10.950807
2           9.21034
3         10.558414
4          9.305651
            ...    
401120     9.259131
401121     9.305651
401122     9.350102
401123      9.10498
401124     8.955448
Name: SalePrice, Length: 401125, dtype: Float64

In [20]:
display_all(df_raw.describe(include = 'all').T)

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
SalesID,401125.0,<NA>,<NA>,<NA>,1919712.521503,1139246.0,1418371.0,1639422.0,2242707.0,6333342.0,909021.492667
SalePrice,401125.0,<NA>,<NA>,<NA>,10.103096,8.4659,9.581904,10.085809,10.596635,11.863582,0.693621
MachineID,401125.0,<NA>,<NA>,<NA>,1217902.517971,0.0,1088697.0,1279490.0,1468067.0,2486330.0,440991.954249
ModelID,401125.0,<NA>,<NA>,<NA>,6889.70298,28.0,3259.0,4604.0,8724.0,37198.0,6221.777842
datasource,401125.0,<NA>,<NA>,<NA>,134.66581,121.0,132.0,132.0,136.0,172.0,8.962237
auctioneerID,380989.0,<NA>,<NA>,<NA>,6.55604,0.0,1.0,2.0,4.0,99.0,16.976779
YearMade,401125.0,<NA>,<NA>,<NA>,1899.156901,1000.0,1985.0,1995.0,2000.0,2013.0,291.797469
MachineHoursCurrentMeter,142765.0,<NA>,<NA>,<NA>,3457.955353,0.0,0.0,0.0,3025.0,2483300.0,27590.256413
UsageBand,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,NaN,NaN,NaN,2004-08-02 07:47:13.986413184,1989-01-17 00:00:00,2000-10-07 00:00:00,2006-03-23 00:00:00,2009-03-24 00:00:00,2011-12-30 00:00:00,NaN


In [21]:
# same as 
# df_raw.SalePrice
df_raw['SalePrice']

0          11.09741
1         10.950807
2           9.21034
3         10.558414
4          9.305651
            ...    
401120     9.259131
401121     9.305651
401122     9.350102
401123      9.10498
401124     8.955448
Name: SalePrice, Length: 401125, dtype: Float64

In [22]:
# to convert back prices to original values
np.exp(df_raw.SalePrice)

0         66000.0
1         57000.0
2         10000.0
3         38500.0
4         11000.0
           ...   
401120    10500.0
401121    11000.0
401122    11500.0
401123     9000.0
401124     7750.0
Name: SalePrice, Length: 401125, dtype: Float64

<a id="naive_training"></a>

### $\bullet$ Naive first model training

[Back to top](#plan)

In [6]:
model = RandomForestRegressor(n_jobs = -1)

In [7]:
# The following code is supposed to fail due to string values in the input data
model.fit(df_raw.drop('SalePrice', axis = 1), df_raw.SalePrice)

NameError: name 'df_raw' is not defined

This dataset contains a mix of **continuous** and **categorical** variables.

The following method extracts particular date fields from a complete datetime for the purpose of constructing categoricals.  You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

<a id="convert_date"></a>

### $\bullet$ Convert date times

[Back to top](#plan)

In [25]:
df_raw.fiModelDesc.value_counts()

fiModelDesc
310G              5039
416C              4869
580K              4315
310E              4233
140G              4083
                  ... 
PC220LC-7GALEO       1
LX885T               1
LS120                1
SE240NLC-3           1
TL210                1
Name: count, Length: 4999, dtype: Int64

In [26]:
df_raw.saledate

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

In [27]:
# date-time object was created within the df with parse_dates=["saledate"]
df_raw.saledate.dt

In [28]:
# add_datepart adds new columns to the dataframe
df_transformed = add_date_columns(df_raw, fields = ['saledate'])

In [29]:
# comparaison of dataframe shape before & after applying add_datepart
print(df_raw.shape)
print(df_transformed.shape)

(401125, 53)
(401125, 64)


In [30]:
df_transformed.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int32

In [31]:
df_transformed.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleDay', 'saleDayofweek', 'saleDayofyear', 'saleIs_mont

In [32]:
df_transformed.saleDayofweek

0         3
1         4
2         3
3         3
4         3
         ..
401120    2
401121    2
401122    2
401123    1
401124    1
Name: saleDayofweek, Length: 401125, dtype: int32

<a id="convert_strings"></a>

### $\bullet$ Convert strings into numerotated categories

[Back to top](#plan)

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we convert strings to pandas categories.

In [33]:

df_transformed = change_columns_from_str_to_categorical(df_transformed)

We can specify the order to use for categorical variables if we wish:

In [34]:
df_transformed.UsageBand

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120      <NA>
401121      <NA>
401122      <NA>
401123      <NA>
401124      <NA>
Name: UsageBand, Length: 401125, dtype: category
Categories (3, string): [High < Low < Medium]

In [35]:
df_transformed.UsageBand.cat.categories.tolist()

['High', 'Low', 'Medium']

In [36]:
df_transformed.UsageBand.dtype

CategoricalDtype(categories=['High', 'Low', 'Medium'], ordered=True, categories_dtype=string)

In [37]:
df_transformed.UsageBand = df_transformed.UsageBand.cat.set_categories(['Low', 'Medium', 'High'], ordered = True)

In [38]:
df_transformed.UsageBand.cat.categories.tolist()

['Low', 'Medium', 'High']

Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. Optionally, we can replace the text categories with numbers, which will make this variable non-categorical, like so:.

In [39]:
df_transformed.UsageBand

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']

In [40]:
df_transformed.UsageBand.cat.categories

Index(['Low', 'Medium', 'High'], dtype='object')

In [41]:
df_transformed.UsageBand.cat.codes

0         0
1         0
2         2
3         2
4         1
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

In [42]:
# This step overwrites the raw categories with integers
df_transformed.UsageBand = df_transformed.UsageBand.cat.codes

In [43]:
# - values corresponds to NaN values in the original column
df_transformed.UsageBand

0         0
1         0
2         2
3         2
4         1
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Name: UsageBand, Length: 401125, dtype: int8

<a id="nans"></a>

### $\bullet$ Inspect missing values

[Back to top](#plan)

We're still not quite done - for instance we have lots of missing values, which we can't pass directly to a random forest.

In [44]:
df_transformed.isnull().describe()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_start,saleIs_month_end,saleIs_quarter_start,saleIs_quarter_end,saleIs_year_start,saleIs_year_end,saleElapsed
count,401125,401125,401125,401125,401125,401125,401125,401125,401125,401125,...,401125,401125,401125,401125,401125,401125,401125,401125,401125,401125
unique,1,1,1,1,1,2,1,2,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,401125,401125,401125,401125,401125,380989,401125,258360,401125,401125,...,401125,401125,401125,401125,401125,401125,401125,401125,401125,401125


In [45]:
display_all(df_transformed.isnull().describe().T)

,count,unique,top,freq
SalesID,401125,1,False,401125
SalePrice,401125,1,False,401125
MachineID,401125,1,False,401125
ModelID,401125,1,False,401125
datasource,401125,1,False,401125
auctioneerID,401125,2,False,380989
YearMade,401125,1,False,401125
MachineHoursCurrentMeter,401125,2,True,258360
UsageBand,401125,1,False,401125
fiModelDesc,401125,1,False,401125


In [46]:
# display amount of missing values per column :
# column name              ratio of nans
display_all(df_transformed.isnull().sum().sort_index()/len(df_transformed))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

<a id="save_data"></a>

### $\bullet$ Save preprocessed data

[Back to top](#plan)

But let's save this file for now, since it's already in format can we be stored and accessed efficiently.

In [47]:
# 18 MB
df_transformed.to_feather(path_to_data / 'transformed')

In [48]:
# 140 MB !!
df_transformed.to_csv(path_to_data / 'transformed.csv', index = False)

<a id="numericalize_data"></a>

### $\bullet$ Fully numericalize data

[Back to top](#plan)

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [49]:
df_transformed = pd.read_feather(path_to_data / 'transformed')

In [50]:
df_processed, y, nas = process_df(df_transformed, y_field = 'SalePrice')

In [51]:
df_processed.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'fiModelDesc',
       'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor',
       'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup',
       'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type',
       'Ride_Control', 'Stick', 'Transmission', 'Turbocharged',
       'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower',
       'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control',
       'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks',
       'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width',
       'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type',
       'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleDay', 'saleDayofweek', 'saleDayofyear', 'saleIs_month_start',
   

In [52]:
df_processed.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,fiBaseModel,...,saleDayofyear,saleIs_month_start,saleIs_month_end,saleIs_quarter_start,saleIs_quarter_end,saleIs_year_start,saleIs_year_end,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,999089,3157,121,3,2004,68,0,950,296,...,320,False,False,False,False,False,False,1163635200,False,False
1,1139248,117657,77,121,3,1996,4640,0,1725,527,...,86,False,False,False,False,False,False,1080259200,False,False
2,1139249,434808,7009,121,3,2001,2838,2,331,110,...,57,False,False,False,False,False,False,1077753600,False,False
3,1139251,1026470,332,121,3,2001,3486,2,3674,1375,...,139,False,False,False,False,False,False,1305763200,False,False
4,1139253,1057373,17311,121,3,2007,722,1,4208,1529,...,204,False,False,False,False,False,False,1248307200,False,False


In [53]:
y

<FloatingArray>
[11.097410021008562, 10.950806546816688,  9.210340371976184,
 10.558413520275876,  9.305650551780507, 10.184900011974314,
   9.95227771670556, 10.203592144986466,  9.975808214115753,
 11.082142548877775,
 ...
  9.210340371976184,  9.047821442478408,  9.210340371976184,
  9.159047077588632,  9.047821442478408,  9.259130536145614,
  9.305650551780507,  9.350102314351341,  9.104979856318357,
  8.955448122347393]
Length: 401125, dtype: Float64

In [54]:
nas

{'auctioneerID': np.float64(2.0), 'MachineHoursCurrentMeter': np.float64(0.0)}

We now have something we can pass to a random forest!

<a id='bottom'></a>

[Back to top](#plan)